### This file will basically import pretrained model weights and perform prediction on a folder of files.

In [ ]:
import numpy as np
import os
import pandas as pd
import sklearn
from  sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
import glob
from sklearn.metrics import mean_squared_error
import neurokit2 as nk
import joblib

# !pip install neurokit2 joblib

### Load model to predict arousal

In [ ]:
regressionTree_asl = joblib.load("path/to/pretrained/model/weights/for/arousal")    #weights are given in our github repo, whose path could be found from readme

### Load model to predict velence

In [ ]:
regressionTree_vel = joblib.load("path/to/pretrained/model/weights/for/velence")    #weights are given in our github repo, whose path could be found from readme

### Data Cleaning, Feature extraction and Testing 

In [ ]:
directory = "path/to/folder/of/test/csvs" + "/"                                    #folder of csvs,folder of test physiology csvs in our case
directory_to_store = "path/where/you/want/to/save/output/predictions" + "/"        #output folder path
csv_files = glob.glob(directory)
csvname = os.listdir(directory)

In [ ]:

for csv in csvname:

    df = pd.read_csv(directory + csv)

    #data cleaning and feature extraction using neurokit library
    temp_ecg= nk.ecg_process(df.ecg,1000)
    temp_bvp= nk.ppg_process(df.bvp,1000)
    temp_rsp= nk.rsp_process(df.rsp,1000)
    temp_gsr= nk.eda_process(df.gsr,1000)

    temp_emg_coru= nk.emg_process(df.emg_coru,1000)
    temp_emg_coru[0].EMG_Amplitude*=100     

    temp_emg_zygo= nk.emg_process(df.emg_zygo,1000)
    temp_emg_zygo[0].EMG_Amplitude*=100 

    temp_emg_trap= nk.emg_process(df.emg_trap,1000)
    temp_emg_trap[0].EMG_Amplitude*=100   

    df.skt.fillna(method='pad')
    temp_skt=nk.ppg_process(df.skt,1000)
    temp_skt[0].PPG_Clean+=df.skt.mean() 

    df.insert(2, "Heart_Rate",temp_ecg[0].ECG_Rate, True)
    df.insert(4, "BVP_clean",temp_bvp[0].PPG_Clean, True)
    df.insert(6, "GSR_clean_phasic",temp_gsr[0].EDA_Phasic, True)
    df.insert(8, "RR_clean",temp_rsp[0].RSP_Rate, True)
    df.insert(10, "skt_clean",temp_skt[0].PPG_Clean, True)
    df.insert(12, "zygo_clean",temp_emg_zygo[0].EMG_Amplitude, True)
    df.insert(14, "coru_clean",temp_emg_coru[0].EMG_Amplitude, True)
    df.insert(16, "trap_clean",temp_emg_trap[0].EMG_Amplitude, True)
    

    #removing first 10 second and last 10 second data, but we have trained model on 2 second delay because as per our findings it gives better results because humans delay to respond in joystick
    #so we remove first 8 seconds and last 12 seocnds data
    
    df = df.iloc[8000:-12000,:]
    df = df.reset_index(drop=True)
    df = df.drop(columns = ["time","ecg","bvp","gsr","rsp","skt","emg_zygo","emg_coru","emg_trap"], axis = 1)   #dropping unnecessary columns
  
    #down sampling our data to 200Hz
    array = range(0, len(df),5) 
    df = df.iloc[array]
    df = df.reset_index(drop=True)   
    
    #prediction of valence and arousal using loaded models
    arousal = regressionTree_asl.predict(df)
    valence = regressionTree_vel.predict(df)
    
    # storing values in desired path, which is "directory_to_store"
    p_df = pd.DataFrame({"arousal":arousal,"valence":valence})
    array1 = range(0, len(p_df),10)
    df1 = p_df.iloc[array1]
    df1 = df1.reset_index(drop=True) 
    df1.insert(0,"time",range(10000,10000+(len(df1)*50),50))
    df1["arousal"] = round(df1["arousal"],3)
    df1["valence"] = round(df1["valence"],3)
    df1.to_csv(directory_to_store + csv,index = False)

  